<a href="https://colab.research.google.com/github/pea-sys/Til/blob/master/NumbaPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 概要
NumbaはPythonコードの選択部分をネイティブコードにコンパイルすることができるパッケージ。   
計算を高速化することで、今後の開発時間を効率的に使用できるようになる。  
今後のために押さえておきたい知識。
## A simple example
まずはNumba未使用で計算時間を測定

In [0]:
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [0]:
import numpy as np

original = np.arange(0.0, 10.0, 0.01, dtype='f4')
shuffled = original.copy()
np.random.shuffle(shuffled)

In [19]:
sorted = shuffled.copy()
bubblesort(sorted)
print(np.array_equal(sorted, original))

True


In [20]:
sorted[:] = shuffled[:]
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

1 loop, best of 3: 296 ms per loop


In [21]:
%timeit sorted[:] = shuffled[:]

The slowest run took 34.11 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 702 ns per loop


## Compiling a function with numba.jit using an explicit function signature
Numba使用して計算時間を測定

In [22]:
print(numba.jit.__doc__)


    This decorator is used to compile a Python function into native code.

    Args
    -----
    signature:
        The (optional) signature or list of signatures to be compiled.
        If not passed, required signatures will be compiled when the
        decorated function is called, depending on the argument values.
        As a convenience, you can directly pass the function to be compiled
        instead.

    locals: dict
        Mapping of local variable names to Numba types. Used to override the
        types deduced by Numba's type inference engine.

    target: str
        Specifies the target platform to compile for. Valid targets are cpu,
        gpu, npyufunc, and cuda. Defaults to cpu.

    pipeline_class: type numba.compiler.BasePipeline
            The compiler pipeline type for customizing the compilation stages.

    options:
        For a cpu target, valid options are:
            nopython: bool
                Set to True to disable the use of PyObjects and Python 

In [23]:
!pip install numba

In [0]:
import numba
bubblesort_jit = numba.jit("void(f4[:])")(bubblesort)

In [25]:
sorted[:] = shuffled[:] # reset to shuffled before sorting
bubblesort_jit(sorted)
print(np.array_equal(sorted, original))

True


In [26]:
%timeit sorted[:] = shuffled[:]; bubblesort_jit(sorted)

1000 loops, best of 3: 1.18 ms per loop


In [27]:
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

1 loop, best of 3: 307 ms per loop


In [0]:
@numba.jit("void(f4[:])")
def bubblesort_jit(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

## Signature
高速コードを生成するために、コンパイラはコードの型情報を必要とする

Some sample signatures follow:


signature | meaning
--- | ---
void(f4[:], u8) | 単精度浮動小数点数の1次元配列と64ビット符号なし整数をとる、戻り値のない関数
i4(f8) | 倍精度浮動小数点数を引数として使用して、32ビット符号付き整数を返す関数
void(f4[:,:],f4[:,:]) | 引数として2つの2次元配列をとる戻り値のない関数

## Compiling a function without providing a function signature 
ほとんどの場合、署名なしでjitを使用するのが最も簡単な方法

In [0]:
bubblesort_autojit = numba.jit(bubblesort)

In [32]:
%timeit sorted[:] = shuffled[:]; bubblesort_autojit(sorted)

The slowest run took 115.33 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.02 ms per loop


## Some extra remarks
numbaについて知っておくと良いこと
* コンパイルに時間がかかります。幸いなことに、特に小さな関数の場合はそれほど時間はかかりません。
* すべてのコードが同等にコンパイルされているわけではありません。 numbaが効率的なネイティブ関数にコンパイルされるコードがあります。生成されたコードがPythonオブジェクトシステムとそのディスパッチセマンティクスにフォールバックしなければならない場合があります。
* 高速ネイティブコード - 「nopython」とも呼ばれます。コンパイラは関数内のすべての型を推測できたので、Pythonランタイムを利用せずにコードを高速のネイティブルーチンに変換できます。
Pythonランタイムを呼び出すネイティブコード - オブジェクトモードとも呼ばれます - コンパイラはすべての型を推測できなかったため、ある時点で値が一般的な「オブジェクト」として型指定されました。つまり、フルネイティブ版は使用できません。代わりに、numbaはPythonのランタイムを使ってコードを生成しますが、それは実際の解釈より速いはずですが、あなたが完全なネイティブ関数から期待できるものとは全くかけ離れています。
デフォルトでは、 'cpu'ターゲットは 'nopython'モードで関数をコンパイルしようとします。これが失敗すると、オブジェクトモードで再試行します。

この例では、Pythonオブジェクトにフォールバックすると、生成されたコードがどのように遅くなるかを示します。

In [0]:
@numba.jit("void(i1[:])")
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

from decimal import Decimal
@numba.jit("void(i1[:])")
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

res = np.zeros((10000,), dtype="i1")

In [37]:
%timeit test(res)

The slowest run took 10.89 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 21.4 µs per loop


In [38]:
%timeit test2(res)

The slowest run took 17.67 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 6.11 ms per loop


nopythonのコード生成が失敗した場合、失敗を強制することが可能です。  
これにより、Pythonランタイムに依存しない、特定の関数のコードを生成することが可能かどうかについてフィードバックを得ることができます。  
オブジェクトモードではパフォーマンスが大幅に低下する可能性があるため、これは高速コードを作成しようとするときに役立ちます。

In [0]:
@numba.jit("void(i1[:])", nopython=True)
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

一方、nopythonキーワードを渡すとtest2は失敗します。

In [42]:
@numba.jit("void(i1[:])", nopython=True)
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

TypingError: ignored